In [1]:
import tkinter as tk  
from tkinter import *
from tkinter import font,messagebox
from functools import partial  
import pandas as pd
import datetime
from pymongo import MongoClient
import warnings; warnings.simplefilter('ignore')
import cv2
import matplotlib.pyplot as plt
from PIL import *
from PIL import Image, ImageTk

now = datetime.datetime.now()

In [2]:
from pymongo import MongoClient

client = MongoClient()
client = MongoClient("mongodb://localhost:27017/")
mydb = client["StudentAttendance"]
curAtt = mydb["Attendance"] # for storing the current attendance
stData = mydb["studentData"] #it contains all student details

In [3]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from PIL import ImageGrab

path = "...\\Train Image"
images = []
classNames = []
myList = os.listdir(path)
#print(myList)
global encodeListKnown
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

#print(classNames)

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

encodeListKnown = findEncodings(images)
print("Encoding Complete")



Encoding Complete


In [3]:
import datetime

#CReating first windows of interface
root = Tk()
#Providing thee title of the windows box and setting the geometry
root.title("Attendance Management")
root.geometry("600x400")
root.resizable(width=False, height=False)
msg = "no"


def Attendance():
    top = Toplevel(root)
    top.geometry("400x400")
    #Setting the window to be fixed and providing the title
    top.resizable(width=False, height=False)
    top.title("Attendance Managment")
    top.wm_transient(root)
    
    
    def manualDetails():
        manDetails = Toplevel(root)
        manDetails.geometry("450x350")
        manDetails.resizable(width=False, height=False)
        top.destroy()
        manDetails.wm_transient(root)
        def name_focus_in(_):
            name.delete(0, tk.END)
            name.config(fg='black')
        
        def sclass_focus_in(_):       
            cl.delete(0, tk.END)
            cl.config(fg='black')
        def id_focus_in(_):      
            Ids.delete(0, tk.END)
            Ids.config(fg='black')
#         def roll_focus_in(_): 
#             roll.delete(0, tk.END)
#             roll.config(fg='black')

        def reset():
                #roll_focus_in(_)
                id_focus_in(_)
                sclass_focus_in(_)
                name_focus_in(_)    
                
        def saveToDB(stid,stdname,stdclass):
                global msg
                err_msg = ""
                sid = stid.get()
                
                #sroll = stdroll.get()
                
                
                
                if(sid!=""):
                    cur = stData.find_one({"Student ID":int(sid)})
                    
                    if (cur != None):
                        reset()
                        Ids.insert(0,cur["Student ID"])
                        name.insert(0,cur["Name"])
                        cl.insert(0,cur["Class"])
                        #roll.insert(0,cur["Roll"])                        
                        
                    else:      
                        reset()
                        err_msg = messagebox.showerror("Not Found","Not Found, Try Again !") 
                        
                   
                    if(err_msg!="ok"):
                        msg = messagebox.askquestion("Confirm","Are you sure?")
                        now = datetime.datetime.now()
                        stime = now.strftime("%Y-%m-%d %H:%M:%S")
                        if(msg=="yes"):
                            sname = stdname.get()
                            sclass = stdclass.get()
                            curAtt.insert_one({"ID":sid,"Name":sname,"Class":sclass,"Date & Time":stime})
                            plabel = Label(root,text = "Attendance Marked !")
                            plabel.place(x = 230, y = 280)
                            manDetails.destroy()
                            top.destroy()   
                    
                else:
                    #raise Exception("Enter All Details")
                    Label(manDetails,text = "Enter All Details",fg = "red").place(x = 280,y = 280)
                    
                    
      
        stid = StringVar()
        stdname= StringVar()
        stdclass = StringVar()
        stdroll = StringVar()
        

        Ids = Entry(manDetails, textvariable=stid,font="Arial",cursor="xterm",justify="center",bd=2,bg="white",fg="black",width = 25)
        Ids.place(x=150,y=30)
        Label(manDetails, text = "ID :-",font = ("Fixedsys",15) ).place(x = 40, y = 35)
        
        name = Entry(manDetails, textvariable=stdname,font="Arial",cursor="xterm",justify="center",bd=2,bg="white",fg="black",width = 25)
        name.place(x=150,y=80)
        Label(manDetails, text = "Name :-",font = ("Fixedsys",15)).place(x = 40, y = 85)
        

        cl = Entry(manDetails, textvariable=stdclass,font="Arial",cursor="xterm",justify="center",bd=2,bg="white",fg="black",width = 25)
        cl.place(x=150,y=130)
        Label(manDetails, text = "Class :-",font = ("Fixedsys",15) ).place(x = 40, y = 135)
               
        
#         roll = Entry(manDetails, textvariable=stdroll,font="Arial",cursor="xterm",justify="center",bd=2,bg="white",fg="black")
#         roll.place(x=150,y=180)
#         Label(manDetails, text = "Roll :-",font = ("Fixedsys",15) ).place(x = 40, y = 185)
                
        saveToDB = partial(saveToDB,stid,stdname,stdclass)
        
        submit_button = Button(manDetails, text = "Submit", command = saveToDB,width = 15)
        submit_button.place(x = 120,y = 200)  
        manDetails.bind("<Return>",lambda event=None: submit_button.invoke()) # bind to Enter button

        reset_button = Button(manDetails, text = "Clear", command = reset,width = 15).place(x = 260,y = 200)
        


        manDetails.mainloop()
        
    def faceRecog():        
        def markAttendance(ID):
            cur = stData.find_one({"Student ID":int(ID)})
            if(cur!=None):
                name = cur["Name"]
                sclass = cur["Class"]
                now = datetime.datetime.now()
                stime = now.strftime("%Y-%m-%d %H:%M:%S")
                curAtt.insert_one({"ID":ID,"Name":name,"Class":sclass,"Date & Time":stime})
                messagebox.showinfo("Success","Attendance Marked !")
            else:
                messagebox.showerror("Record","Record Not found")
                
                
        face = Toplevel(root)
        face.geometry("400x400")
        #Setting the window to be fixed and providing the title
        face.resizable(width=False, height=False)
        face.title("Attendance Managment")
        face.wm_transient(root)
        
        width, height = 600, 600
        face.bind('<Escape>', lambda e: face.quit())
        lmain = tk.Label(face)
        lmain.pack()

        def show_frame(frame,sid):
            cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
            img = Image.fromarray(cv2image)
            imgtk = ImageTk.PhotoImage(image=img)
            lmain.imgtk = imgtk
            lmain.configure(image=imgtk)
            lmain.after(10, show_frame)    
            markAttendance(sid)         
            face.after(5000,lambda:face.destroy())        
            
        cap = cv2.VideoCapture(0)
        ID = 0
        success, img = cap.read()
        

        if success:
            imgS = cv2.resize(img, (0,0), None, 0.25,0.25)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

            faceCurFrame = face_recognition.face_locations(imgS)
            encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)

            for encodeFace, (top, right, bottom, left) in zip(encodeCurFrame, faceCurFrame):
                matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
                faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

                matchIndex = np.argmin(faceDis)

                if matches[matchIndex]:
                    ID = classNames[matchIndex]

                print("Name",ID)

                # Scale back up face locations since the frame we detected in was scaled to 1/4 size
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                # Draw a box around the face
                cv2.rectangle(img, (left+10, top+10), (right+10, bottom+10), (0, 0, 255), 2)
                show_frame(img,ID)              
                
                cv2.waitKey(1)
                
        
        # Release handle to the webcam
        cv2.destroyAllWindows()
        cap.release()    
        face.mainloop()


         
    
    
    manual_button = Button(top,text = "Manual",command = manualDetails,pady=10, width = 20,font = ("Fixedsys",12))
    manual_button.place(x = 120, y = 130)
    
    facial_button = Button(top,text = "Facial Recognition",command = faceRecog,pady=10, width = 20,font = ("Fixedsys",12))
    facial_button.place(x = 120, y = 200)
    
    top.mainloop()
#-------------------------------------------------------------------------------------------------------------------------------    
def login():
        log = Toplevel(root)
        log.title("Attendance Management")
        log.geometry("600x400")
        log.resizable(width=False, height=False)   
        def add_data():
                add=Toplevel(root)
                add.title("Attendance Management")
                add.geometry("600x400")
                add.resizable(width=False, height=False)
                
                def reset():
                    name.delete(0,tk.END)
                    fname.delete(0,tk.END)
                    age.delete(0,tk.END)
                    cl.delete(0,tk.END)
                    section.delete(0,tk.END)
                    address.delete(0,tk.END)
                    email.delete(0,tk.END)
                    mno.delete(0,tk.END)
                    stID.delete(0,tk.END)
                
                
                def newdata(name,Father,Age,Sclass,Section,Address,Email,Mobile,Id):
                    name=name.get()
                    father = Father.get()
                    age=Age.get()
                    Sec=Section.get()
                    cl = Sclass.get()
                    Add=Address.get()
                    email=Email.get()
                    mob=Mobile.get()
                    sID = Id.get()
                    stData.insert_one({"Student ID":int(sID),"Name":name,"Father":father,"Class":cl,"Age":age,"Section":Sec,"Address":Add,
                                       "Email":email,"Mobile":mob})
                    messagebox.showinfo("Record","Data Added Successfully !")
                    add.destroy()

               
                name=StringVar() 
                Father =StringVar() 
                Age=StringVar()
                Sclass=StringVar()
                Section=StringVar()
                Address=StringVar()
                Email=StringVar()
                Mobile=StringVar()
                Id = StringVar()


                Label(add,text="Student ID").place(x=150,y=75)
                stID = Entry(add,textvariable=Id, width = 30)
                stID.place(x=250,y=75)
                
                Label(add,text="Full Name").place(x=150,y=100)
                name = Entry(add,textvariable=name, width = 30)
                name.place(x=250,y=100)

                Label(add,text="Father's Name").place(x=150,y=125)
                fname = Entry(add,textvariable=Father, width = 30)
                fname.place(x=250,y=125)

                Label(add,text="Age").place(x=150,y=150)
                age = Entry(add,textvariable=Age, width = 30)
                age.place(x=250,y=150)

                Label(add,text="class").place(x=150,y=175)
                cl = Entry(add,textvariable=Sclass, width = 30)
                cl.place(x=250,y=175)

                Label(add,text="section").place(x=150,y=200)
                section = Entry(add,textvariable=Section, width = 30)
                section.place(x=250,y=200)

                Label(add,text="Address").place(x=150,y=225)
                address = Entry(add,textvariable=Address, width = 30)
                address.place(x=250,y=225)

                Label(add,text="Email address").place(x=150,y=250)
                email = Entry(add,textvariable=Email, width = 30)
                email.place(x=250,y=250)

                Label(add,text="Mobile number").place(x=150,y=275)
                mno = Entry(add,textvariable=Mobile, width = 30)
                mno.place(x=250,y=275)
                
            
                newdata=partial(newdata,name,Father,Age,Sclass,Section,Address,Email,Mobile,Id)
                submit_button = Button(add,text="submit",command=newdata)
                submit_button.place(x=250,y=325)
                add.bind("<Return>",lambda event=None: submit_button.invoke())
                Button(add,text="Clear",command=reset).place(x=300,y=325)

                add.mainloop()
#-------------------------------------------------------------------------------------------------------------------------------                
        def update_rec():
            update = Toplevel(root)
            update.title("Attendance Management")
            update.geometry("600x400")
            update.wm_transient(root)
            update.resizable(width=False, height=False)            
            
            Id = StringVar()
            
            def get_records(Id):
                name=StringVar() 
                Father =StringVar() 
                Age=StringVar()
                Sclass=StringVar()
                Section=StringVar()
                Address=StringVar()
                Email=StringVar()
                Mobile=StringVar()
                
                sid = Id.get()
                cur = stData.find_one({"Student ID":int(sid)})
                
                def exit():
                    update.destroy()
                
                def updatetoDB(name,Father,Age,Sclass,Section,Address,Email,Mobile,Id):
                    name=name.get()
                    father = Father.get()
                    age=Age.get()
                    Sec=Section.get()
                    cl = Sclass.get()
                    Add=Address.get()
                    email=Email.get()
                    mob=Mobile.get()
                    sID = Id.get()
                    
                    stData.update_one({"Student ID":int(sID)},{"$set":{"Name":name}})
                    stData.update_one({"Student ID":int(sID)},{"$set":{"Father":father}})
                    stData.update_one({"Student ID":int(sID)},{"$set":{"Age":age}})
                    stData.update_one({"Student ID":int(sID)},{"$set":{"Section":Sec}})
                    stData.update_one({"Student ID":int(sID)},{"$set":{"Class":cl}})
                    stData.update_one({"Student ID":int(sID)},{"$set":{"Address":Add}})
                    stData.update_one({"Student ID":int(sID)},{"$set":{"Email":email}})
                    stData.update_one({"Student ID":int(sID)},{"$set":{"Mobile":mob}})

                    messagebox.showinfo("Record","Updated Successfully !")
                    update.destroy()

                

                if(cur != None):           

                    Label(update,text="Full Name").place(x=150,y=100)
                    name = Entry(update,textvariable=name, width = 30)
                    name.place(x=250,y=100)
                    name.insert(0,cur["Name"])

                    Label(update,text="Father's Name").place(x=150,y=125)
                    fname = Entry(update,textvariable=Father, width = 30)
                    fname.place(x=250,y=125)
                    fname.insert(0,cur["Father"])

                    Label(update,text="Age").place(x=150,y=150)
                    age = Entry(update,textvariable=Age, width = 30)
                    age.place(x=250,y=150)
                    age.insert(0,cur["Age"])

                    Label(update,text="class").place(x=150,y=175)
                    cl = Entry(update,textvariable=Sclass, width = 30)
                    cl.place(x=250,y=175)
                    cl.insert(0,cur["Class"])

                    Label(update,text="section").place(x=150,y=200)
                    section = Entry(update,textvariable=Section, width = 30)
                    section.place(x=250,y=200)
                    section.insert(0,cur["Section"])

                    Label(update,text="Address").place(x=150,y=225)
                    address = Entry(update,textvariable=Address, width = 30)
                    address.place(x=250,y=225)
                    address.insert(0,cur["Address"])

                    Label(update,text="Email address").place(x=150,y=250)
                    email = Entry(update,textvariable=Email, width = 30)
                    email.place(x=250,y=250)
                    email.insert(0,cur["Email"])

                    Label(update,text="Mobile number").place(x=150,y=275)
                    mno = Entry(update,textvariable=Mobile, width = 30)
                    mno.place(x=250,y=275)
                    mno.insert(0,cur["Mobile"])  
                    
                    updatetoDB = partial(updatetoDB,name,Father,Age,Sclass,Section,Address,Email,Mobile,Id)
                    submit_button = Button(update,text="Update",width = 10,command = updatetoDB)
                    submit_button.place(x=250,y=325)
                    update.bind("<Return>",lambda event=None: submit_button.invoke())
                    Button(update,text="Exit",command = exit,width = 10).place(x=340,y=325)
                    
                else:
                    messagebox.showerror("Error","Record Not Found")
                    update.destroy()
                    update_rec()                   
                
           
            Label(update,text="Student ID").place(x=150,y=75)
            Entry(update,textvariable=Id, width = 30).place(x=250,y=75)
            
            get_records = partial(get_records,Id)
            Button(update,text = "Search", command = get_records).place(x = 450,y=70)
            
            

        def delete_rec():
            delete = Toplevel(root)
            delete.title("Attendance Management")
            delete.geometry("600x400")
            delete.wm_transient(root)
            delete.resizable(width=False, height=False)
            
            Id = StringVar()
            
            def delData(Id):
                sid = Id.get()
                cur = stData.find_one({"Student ID":int(sid)})
                if(cur!=None):
                    msg = messagebox.askquestion("Confirm","Are you Sure ?")
                    if(msg == "yes"):
                            stData.delete_one({"Student ID":int(sid)})
                            messagebox.showinfo("Deleted","Record Deleted !")
                            delete.destroy()
                else:
                    messagebox.showerror("Record","Record Not Found !")
                        
                

            Label(delete,text="Student ID").place(x=150,y=75)
            Entry(delete,textvariable=Id, width = 30).place(x=250,y=75)
            
            delData = partial(delData, Id)

            Button(delete,text = "Delete", command = delData).place(x = 450,y=70)
            delete.mainloop()



               
        def main_login(username, password):            
            usname = username.get()
            upass = password.get()
            if((usname == "admin") and (upass == "admin")):
                mlog = Toplevel(root)
                mlog.title("Attendance Management")
                mlog.geometry("600x400")
                mlog.resizable(width=False, height=False)
                mlog.wm_transient(root)
                log.destroy()
                
                add_button = Button(mlog,text = "New",command = add_data,pady=10, width = 15,font = ("Fixedsys",12))
                add_button.place(x = 230, y = 110)
                upd_button = Button(mlog,text = "Show / Update",command = update_rec,pady=10, width = 15,font = ("Fixedsys",12))
                upd_button.place(x = 230, y = 180)
                del_button = Button(mlog,text = "Delete",command = delete_rec,pady=10, width = 15,font = ("Fixedsys",12))
                del_button.place(x = 230, y = 250)
                
                mlog.mainloop()
            else: 
                error = Label(log,text = "Invalid User",font = ("Fixedsys",12),fg = 'red')
                error.place(x = 230,y = 280)
                
                
        
        username = StringVar()
        password = StringVar()
        
        uname = Entry(log, textvariable=username,font="Arial",cursor="xterm",justify="center",bd=2,bg="white",fg="black").place(x=210,y=112)
        ulabel = Label(log, text = "USERNAME",font = ("Fixedsys",15)).place(x = 250, y = 75)
        

        passw = Entry(log, textvariable=password,font="Arial",cursor="xterm",justify="center",bd=2,bg="white",fg="black").place(x=210,y=190)
        plabel = Label(log, text = "PASSWORD",font = ("Fixedsys",15) ).place(x = 250, y = 150)
       
        
        main_login = partial(main_login,username,password)

        submit_button = Button(log, text = "Submit", command = main_login, width = 8)
        submit_button.place(x = 230,y = 230)
        log.bind("<Return>",lambda event=None: submit_button.invoke())


        reset_button = Button(log, text = " Reset ", command = "reset", width = 8)
        reset_button.place(x = 310,y = 230)


        log.mainloop()
#-------------------------------------------------------------------------------------------------------------------------------
att_button = Button(root,text = "Attendance",command = Attendance,pady=10, width = 15,font = ("Fixedsys",12))
att_button.place(x = 230, y = 130)
log_button = Button(root,text = "Login",command = login,pady=10, width = 15,font = ("Fixedsys",12))
log_button.place(x = 230, y = 200)


root.mainloop()

In [26]:
import tkinter as tk
from tkinter import filedialog

def UploadAction(event=None):
    filename = filedialog.askopenfilename()
    print('Selected:', filename)

root = tk.Tk()
button = tk.Button(root, text='Open', command=UploadAction)
button.pack()

root.mainloop()

Selected: C:/Users/kumar/Desktop/off css.png
